# **KUBIG 24S NLP Basic Team 4**

## **사투리 번역기 구현**

### 19기 정종락

# **Import Libraries, Modules, and Dataset**

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/data/'
columns_to_load = ['standard_form', 'dialect_form']

Mounted at /content/drive


In [3]:
jj_train = pd.read_parquet(path + 'jj_train.parquet')

## **Dataset**

In [ ]:
jj_train.shape

(2774257, 9)

In [ ]:
jj_train.tail()

,id,form,standard_form,dialect_form,speaker_id,start,end,note,eojeolList
2774252,DZJD20000080.1.1.125,468.71,473.39,1,지금 우리가 시간이 열 한 시 사십 분이 넘었나? 사십 분 (되난)/(되니까),지금 우리가 시간이 열 한 시 사십 분이 넘었나? 사십 분 되니까,지금 우리가 시간이 열 한 시 사십 분이 넘었나? 사십 분 되난,,"[{'eojeol': '지금', 'id': 1, 'isDialect': False,..."
2774253,DZJD20000080.1.1.126,473.39,477.99,1,으응 사십분 - 내가 삼십 분에 (끝낭)/(끝나서) 사십 분에 통화 하기로 해 가지고,으응 사십분 - 내가 삼십 분에 끝나서 사십 분에 통화 하기로 해 가지고,으응 사십분 - 내가 삼십 분에 끝낭 사십 분에 통화 하기로 해 가지고,,"[{'eojeol': '으응', 'id': 1, 'isDialect': False,..."
2774254,DZJD20000080.1.1.127,477.99,479.40,1,전화 오는 거 (닮은디)/(같은데),전화 오는 거 같은데,전화 오는 거 닮은디,,"[{'eojeol': '전화', 'id': 1, 'isDialect': False,..."
2774255,DZJD20000080.1.1.128,479.40,479.85,2,응~,응~,응~,,"[{'eojeol': '응~', 'id': 1, 'isDialect': False,..."
2774256,DZJD20000080.1.1.129,479.85,482.54,2,사회자님 중간에 역할 하시는 분 어디(가부러시니)/(가버렸니),사회자님 중간에 역할 하시는 분 어디(가부러시니)/(가버렸니),사회자님 중간에 역할 하시는 분 어디(가부러시니)/(가버렸니),,"[{'eojeol': '사회자님', 'id': 1, 'isDialect': Fals..."


In [4]:
jj_train.rename(columns = {'form': 'start',
                           'speaker_id': 'form',
                           'standard_form': 'end',
                           'dialect_form': 'speaker_id',
                           'start': 'standard_form',
                           'end': 'dialect_form'},
                inplace = True)
order = ['id', 'form', 'standard_form',	'dialect_form',
         'speaker_id',	'start',	'end',	'note',	'eojeolList']
jj_train = jj_train[order]

In [5]:
jj_train = jj_train[columns_to_load]

In [6]:
jj_train.head()

,standard_form,dialect_form
0,왜냐하면 이만한 그,왜냐하면 이만한 그
1,-이만- 이만한 크기.,-이만- 이만한 크기.
2,응.,응.
3,제사 때 쓰는 거.,제사 때 쓰는 거.
4,아니 게메 저 차판 자체가 쓸 일이 많지가 않을 거 같아 #이 거기서 #이 왜냐하면,아니 게메 저 차판 자체가 쓸 일이 하지가 않을 거 닮아 이 거기서 이 왜냐하면


In [7]:
jj_train.tail()

,standard_form,dialect_form
2774252,지금 우리가 시간이 열 한 시 사십 분이 넘었나? 사십 분 되니까,지금 우리가 시간이 열 한 시 사십 분이 넘었나? 사십 분 되난
2774253,으응 사십분 - 내가 삼십 분에 끝나서 사십 분에 통화 하기로 해 가지고,으응 사십분 - 내가 삼십 분에 끝낭 사십 분에 통화 하기로 해 가지고
2774254,전화 오는 거 같은데,전화 오는 거 닮은디
2774255,응~,응~
2774256,사회자님 중간에 역할 하시는 분 어디(가부러시니)/(가버렸니),사회자님 중간에 역할 하시는 분 어디(가부러시니)/(가버렸니)


# **Model**

## **표준어 - 제주도, KoBART - Batch Size 32, epoch 5**

In [8]:
jj_train_sampled = jj_train.sample(n = 100000, random_state = 42)

In [9]:
jj_train_sampled

,standard_form,dialect_form
1706384,그니까 요새 다 힘든거라,게난 요새 다 힘든거라
2043808,아 난 &company-name40&,아 난 &company-name40&
1331475,예를 들어 하이에나 사자같은 애들이면은,예를 들어 하이에나 사자같은 애들이면은
1593087,저리로 이렇게 너 나가있어라 하면 나가고,절로 영 너 나강이시라 허민 나가고
2238828,(()) 가스레인지가 있는 게 좀 웃기긴 하지.,(()) 가스레인지가 있는 게 좀 웃기긴 하지.
...,...,...
2071308,아 그래? 부대에 이런게 많아? 그래서,아 기? 부대에 이런게 많아? 그래서
2664871,그 물에 그냥 잠옷 입은 그대로 빠져서 오빠 건져서,그 물에 그냥 잠옷 입은냥 빠져네 오라방 건져네
2419741,골채 해서 이렇게 구멍 막아서,골채 해그냉 영 고망 막아그냉
745600,그런데,그런데


In [10]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터셋 로드
df = jj_train_sampled.copy()

# 필요한 전처리 (예: 결측값 제거 등)
df = df.dropna()

# 학습 및 검증 셋 분리
train_df, val_df = train_test_split(df, test_size = 0.2, random_state = 42)

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

# 토큰화 함수 정의
def tokenize_data(texts):
    return tokenizer(texts, padding = "max_length",
                     max_length = 128, truncation = True, return_tensors = "pt")

# 토큰화 수행
train_encodings = tokenize_data(train_df['standard_form'].tolist())
train_labels = tokenize_data(train_df['dialect_form'].tolist())

val_encodings = tokenize_data(val_df['standard_form'].tolist())
val_labels = tokenize_data(val_df['dialect_form'].tolist())

import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

# 데이터셋 생성
train_dataset = TranslationDataset(train_encodings, train_labels)
val_dataset = TranslationDataset(val_encodings, val_labels)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [11]:
from transformers import BartForConditionalGeneration, AdamW

model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')
optimizer = AdamW(model.parameters(), lr = 2e-5)

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # 데이터 준비
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델 예측 및 손실 계산
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 진행 상황 출력
        loop.set_description(f'Epoch {epoch + 1}')
        loop.set_postfix(loss=loss.item())

    # 검증 단계 (옵션)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    print(f'Validation Loss: {val_loss / len(val_loader)}')

# 모델과 토크나이저 저장 경로
model_save_path = '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_model_st_jj'
tokenizer_save_path = '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_jj'

# 모델 저장
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/2500 [00:00<?, ?it/s]<ipython-input-10-954ca2c0945d>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-10-954ca2c0945d>:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels['input_ids'][idx])
Epoch 1: 100%|██████████| 2500/2500 [17:54<00:

Validation Loss: 0.051286554616689684


Epoch 2: 100%|██████████| 2500/2500 [17:54<00:00,  2.33it/s, loss=0.0402]


...

Validation Loss: 0.03534226689040661


Epoch 5: 100%|██████████| 2500/2500 [17:54<00:00,  2.33it/s, loss=0.0433]


Validation Loss: 0.03551633455902338


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


('/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_jj/tokenizer_config.json',
 '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_jj/special_tokens_map.json',
 '/content/drive/Othercomputers/My MacBook Pro/Colab/KUBIG/24S/NLP/Team/model/translation_tokenizer_st_jj/tokenizer.json')

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

model = BartForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_save_path)
model.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [14]:
# 입력 문장
input_sentence = "저리로 이렇게 너 나가있어라 하면 나가고"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

저리로 영 너 나가서라 하면 나가고


In [15]:
# 입력 문장
input_sentence = "전화 오는 거 같은데"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

전화 오는 거 닮은디


In [16]:
# 입력 문장
input_sentence = "밥 먹었니?"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

밥 먹언?


In [17]:
# 입력 문장
input_sentence = "사회자님 중간에 역할 하시는 분 어디 가버렸니"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

사회자님 중간에 역할 하시는 분 어디 가부러신가


In [18]:
# 입력 문장
input_sentence = "아 그래? 그니까 요새 다 힘든거라 골채 해서 이렇게 구멍 막아서"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

아 기? 그니까 요새 다 힘든거라 골채 해그네 영 구멍 막아그네


In [22]:
# 입력 문장
input_sentence = "그 물에 그냥 잠옷 입은 그대로 빠져서 그 사람을 건져서"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

그 물에 그냥 잠옷 입은 그대로 빠져서 그 사람을 건졍


In [23]:
# 입력 문장
input_sentence = "저리로 이렇게 너 나가있어라 하면 나가고"

# 토큰화
input_ids = tokenizer.encode(input_sentence, return_tensors = "pt").to(device)

# 모델 예측
output_ids = model.generate(input_ids, max_length = 50, num_beams = 5, early_stopping = True)

# 결과 해독
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_sentence)

저리로 영 너 나가서라 하면 나가고
